In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import statistics
import pandas as pd
from scipy import stats
from scipy.stats import norm

# Dados de entrada
df = pd.read_csv("C:/Users/Mariana/Dropbox (Pessoal)/1º sem 2020/MBA/Dados/Prestigio.csv", sep = ',', na_values = '-', encoding= 'unicode_escape')

In [7]:
# Exclui os dados faltantes
df = df.dropna()

# Crias as variáveis dummies para cada categoria de 'type'
# Repare que este comando cria uma dummy para cada categoria
# Assim, posso escolher qual eu quero que seja a categoria referência (não sendo necessariamente a primeira)
df = pd.concat([df, pd.get_dummies(df['type'].astype('category'), prefix = 'd')], axis = 1)
df.describe()

,education,income,women,prestige,census,d_bc,d_prof,d_wc
count,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000
mean,10.795102,6938.857143,28.985714,47.327551,5400.061224,0.448980,0.316327,0.234694
std,2.748937,4228.132538,31.382016,17.094907,2684.302381,0.499947,0.467433,0.425986
min,6.380000,1656.000000,0.000000,17.300000,1113.000000,0.000000,0.000000,0.000000
25%,8.445000,4250.500000,3.267500,35.375000,3115.500000,0.000000,0.000000,0.000000
50%,10.605000,6035.500000,14.475000,43.600000,5131.500000,0.000000,0.000000,0.000000
75%,12.755000,8226.250000,52.202500,59.900000,8328.000000,1.000000,1.000000,0.000000
max,15.970000,25879.000000,97.510000,87.200000,9517.000000,1.000000,1.000000,1.000000


In [3]:
import statsmodels.formula.api as smf

# Função com o método de seleção Forward, usando o R2 ajustado para seleção
# Fonte: https://planspace.org/20150423-forward_selection_with_statsmodels/
def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1 ".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [19]:
# Prepara o arquivo de dados para executar a função forward_select
# tem que excluir as variáveis que não usaremos no modelo para poder rodar a função
# Escolhi blue collar como categoria de referência e, portanto, essa dummy não entrará no modelo
data = df.drop(['occupation', 'census','type','d_bc'], axis=1).astype('float64')

#Executa a função para seleção de variáveis pelo método forward, usando R2 ajustado como critério de seleção
model = forward_selected(data, 'prestige')

# verifica se o banco de dados é o correto
data.head()

,education,income,women,prestige,d_prof,d_wc
0,13.11,12351.0,11.16,68.8,1.0,0.0
1,12.26,25879.0,4.02,69.1,1.0,0.0
2,12.77,9271.0,15.70,63.4,1.0,0.0
3,11.42,8865.0,9.11,56.8,1.0,0.0
4,14.62,8403.0,11.68,73.5,1.0,0.0


In [20]:
# Imprime as variáveis que ficam no modelo final
print(model.model.formula)

prestige ~ education + income + d_prof + d_wc + 1


In [21]:
# Imprime o R2 ajustado do modelo final
print(model.rsquared_adj)

0.8277545132023085


In [27]:
from statsmodels.formula.api import ols

# Ajusta o modelo selecionado pelo método forward com as variáveis> income, education e type
mod = ols('df.prestige ~ df.income + df.education + df.d_prof + df.d_wc',data=df)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            df.prestige   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     117.5
Date:                Sat, 23 May 2020   Prob (F-statistic):           1.70e-35
Time:                        19:13:08   Log-Likelihood:                -328.51
No. Observations:                  98   AIC:                             667.0
Df Residuals:                      93   BIC:                             679.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.6229      5.228     -0.119   

Repare que a variável 'type' parece não ser significativa, mas vamos fazer o teste de hipóteses conjunto para verificar.

In [28]:
# H0: d_prof = d_wc = 0 (são os dois últimos parâmetros de regressão do modelo)
R = [[0, 0, 0, 0, 1], [0, 0, 0, 1, 0]]
print(np.array(R))
print(res.f_test(R))

[[0 0 0 0 1]
 [0 0 0 1 0]]
<F test: F=array([[5.87213149]]), p=0.0039664382934755885, df_denom=93, df_num=2>


Os dois parâmetros de regressão não são conjuntamente diferentes de 0. Assim, vamos excluir apenas um deles do modelo: o que tem o maior p-valor (d_wc)

In [29]:
# Modeolo com: income, education e prof
mod = ols('df.prestige ~ df.income + df.education + df.d_prof',data=df)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            df.prestige   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.827
Method:                 Least Squares   F-statistic:                     156.0
Date:                Sat, 23 May 2020   Prob (F-statistic):           2.24e-36
Time:                        19:13:33   Log-Likelihood:                -329.13
No. Observations:                  98   AIC:                             666.3
Df Residuals:                      94   BIC:                             676.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.5815      4.325      0.597   

Agora todos os parâmetros de regressão são significativamente diferentes de zero (menos o intercepto). Repare que o $R^2$ ajustado e o AIC são quase os mesmos do modelo anterior (adicionado de d_wc) e o BIC diminui um pouco. Por ser um modelo com menos variáveis explicativas, todas significativas, e ter leve indício de ser mais adequado do que o anterior, decido estas são as variáveis explicativas que permaneceráo no modelo final. 
Por questões de melhor interpretação (do intercepto), ajustarei o mesmo modelo com as variáveis quantitativas centradas. Além disso, também verificarei se existe alguma interação entre as variáveis que se mantiveram como significativas.

In [31]:
income_c = data.income-np.mean(data.income)
education_c = data.education-np.mean(data.education)

# para incluir as interações, troco '+' por '*'
mod = ols('df.prestige ~ income_c * education_c * df.d_prof',data=df)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            df.prestige   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.855
Method:                 Least Squares   F-statistic:                     82.76
Date:                Sat, 23 May 2020   Prob (F-statistic):           1.94e-36
Time:                        19:23:13   Log-Likelihood:                -318.44
No. Observations:                  98   AIC:                             652.9
Df Residuals:                      90   BIC:                             673.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

Comecemos com os termos das interações de maior ordem, que são muito difíceis de serem interpretadas. Como a interação de segunda ordem (income:education:df_prof) foi não significativa, começo excluindo-a.

In [33]:
# para incluir as interações, troco '+' por '*'
mod = ols('df.prestige ~ income_c * education_c + income_c * df.d_prof + education_c * df.d_prof',data=df)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            df.prestige   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.853
Method:                 Least Squares   F-statistic:                     94.63
Date:                Sat, 23 May 2020   Prob (F-statistic):           6.32e-37
Time:                        19:30:26   Log-Likelihood:                -319.76
No. Observations:                  98   AIC:                             653.5
Df Residuals:                      91   BIC:                             671.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                46.94

Neste modelo, a interação entre df_prof e education, também não é significativa e irei excluí-la.

In [34]:
mod = ols('df.prestige ~ income_c * education_c + income_c * df.d_prof',data=df)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            df.prestige   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.854
Method:                 Least Squares   F-statistic:                     114.8
Date:                Sat, 23 May 2020   Prob (F-statistic):           5.62e-38
Time:                        19:31:58   Log-Likelihood:                -319.76
No. Observations:                  98   AIC:                             651.5
Df Residuals:                      92   BIC:                             667.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               46.9002 

Income e education também não têm interação significativa.

In [35]:
mod = ols('df.prestige ~ income_c + education_c + income_c * df.d_prof',data=df)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            df.prestige   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.856
Method:                 Least Squares   F-statistic:                     144.9
Date:                Sat, 23 May 2020   Prob (F-statistic):           4.53e-39
Time:                        19:34:15   Log-Likelihood:                -319.80
No. Observations:                  98   AIC:                             649.6
Df Residuals:                      93   BIC:                             662.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             46.9029      1

Agora, todos os parâmetros de regressão são significativamente diferentes de zero e, portanto, este é o modelo final.
Interpretando-o:
1) Intercepto: 46,9 é o valor esperado do prestígio para income e education iguais às médias (6939 e 10,8, respectivamente) e d_prof=0 (incluindo a interação com ela), ou seja, para os tipos de ocupação blue e white collars.

2) Income: 0,0027 é o aumento esperado do prestígio para 1 dólar aumentado na renda, para ocupações com o mesmo tempo de educação e do tipo collar (blue ou white); talvez seja mais interessante interpretar que o aumento esperado do prestígio é de 0,0027*1000 = 2,7 para um aumento de 1000 dólares canadenses na renda de uma ocupação blue ou white collar, por exemplo.

3) Education: espera-se um aumento de 3 unidades no prestígio para o aumento de 1 ano de educação, para ocupações com mesma renda média (income) e de um mesmo tipo (seja ele qual for). Neste caso, não restringimos a interpretação aos blue ou white collar, pois essa variável education não tem interação com tipo de ocupação (type).

4) d_prof: 46,9+8,7 = 55,6 é o valor esperado do prestígio para income e education iguais às médias (6939 e 10,8, respectivamente) e tipo de ocupação profissional (ou gerenciamento ou técnica). Repare que 8,7 é a diferenca no prestígio por se tratar de uma ocupação do tipo profissional (e não blue ou white collor).

5) Interação income e d_prof: 0,0027-0,0020=0,0007 é o aumento esperado do prestígio para 1 dólar aumentado na renda, para ocupações com o mesmo tempo de educação e do tipo profissionais. Repare que esse aumento é menor do que o esperado nas ocupações do tipo blue e white collors. Analogamento ao feito anteriormente, pode-se interpretar como um aumento de 0,7 no prestígio para aumento de 1000 dólares na renda das ocupações do tipo profissionais. 

Todas essas estimativas também estão acompanhadas dos respectivos intervalos de confiança. Repare que o intervalo de confiança para 'd_prof" tem grande amplitude, indicando pouca precisão dessa estimativa.

Podemos confirmar a escolha do modelo final, comparando-se os índices AIC, BIC e $R^2_a$, conforme tabela abaixo. Veja que o modelo escolhido foi o que apresentou os melhors índices (maior $R^2_a$ e menores AIC, BIC).

Modelo | Variáveis                                              | AIC   | BIC   | $$R^2_a$$
-------|--------------------------------------------------------|-------|-------|-------
1      | income, education, type                                | 667   | 679,9 | 82,8%
2      | income, education, prof                                | 666,3 | 676,6 | 82,7%
3      | income, education, prof, todas as interações           | 652,9 | 673,6 | 85,5%
4      | income, education, prof, interações de 1a ordem        | 653,5 | 671,6 | 85,3%
5      | income, education, prof, income*education, income*prof | 651,5 | 667   | 85,4%
6      | income, education, prof, income*prof                   | 649,6 | 662   | 85,6%